# PDR Assessor Data Analysis

In [1]:
#SF Open Data included very incomplete building SF data, so this cleans and analyzes City/County Assessor property tax data

In [2]:
import pandas as pd
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from pandas import ExcelFile

In [3]:
pip install xlrd

You are using pip version 9.0.3, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Accessing 2017-2018 property tax data
url = 'https://www.sfassessor.org/sites/default/files/uploaded/2019.8.12__SF_ASR_Secured_Roll_Data_2017-2018_0.xlsx'

import requests  
r = requests.get(url)  
open('AssessorPropertyData.xls', 'wb').write(r.content)  

32639786

In [5]:
sheet = pd.read_excel('AssessorPropertyData.xls')

In [6]:
pd.set_option("display.max.columns", None)
sheet.head()

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0


In [7]:
#Looking at the data

sheet.sample(n=10)

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL
15038,0000 0690 MARKET ST0304,08I,0311 027,3,TSF,2007,2,2,5,1,0,NaN,NaN,0.0,0.0,1603,0,0.0,NaN,NaN,1000.0,0.08333,NaN,NaN,0,0,17,100528,0,562703,844054,0
94717,0000 0566 CRESTLAKE DR0000,03C,2499A004,18,D,1950,1,2,5,1,1,RH1D,D,0.0,0.0,1271,0,5623.0,NaN,NaN,1000.0,1.00000,NaN,NaN,0,0,17,61117,0,378644,567965,0
146051,0000 2337 25TH ST0000,09E,4281A025A,28,D,1926,1,0,5,1,1,RH2,D,0.0,0.0,878,0,1245.0,NaN,NaN,1000.0,1.00000,NaN,NaN,0,0,17,0,0,311419,467129,0
167650,0000 0120 VALMAR TE0000,10C,5960 005,35,D,1950,1,0,4,1,1,RH1,D,0.0,0.0,750,0,2500.0,R,NaN,1000.0,1.00000,11,NaN,0,7000,17,60609,0,222000,518000,0
62367,0000 4024 CABRILLO ST0000,01E,1603 022,12,D,1931,1,0,5,1,1,RH2,D,0.0,0.0,1475,0,2495.0,NaN,NaN,1000.0,1.00000,NaN,NaN,0,0,17,0,0,35610,37197,0
48154,0000 1260 HAIGHT ST0001,05B,1234 042,9,Z,1928,1,0,4,0,1,RM1,D,37.5,0.0,1049,0,3437.5,R,NaN,1000.0,1.00000,NaN,NaN,0,0,17,161012,0,468000,468000,0
96084,0000 0022 CASTRO ST0000,05J,2611 003,19,D,1906,3,4,10,2,1,RM1,D,90.0,0.0,2798,0,2475.0,R,NaN,1000.0,1.00000,11,NaN,0,7000,17,970825,0,290850,598480,551
39734,0000 2837 WASHINGTON ST0000,07B,1002 045,7,Z,1900,3,3,6,2,1,RH2,D,0.0,0.0,2693,0,3514.0,O,NaN,1000.0,1.00000,NaN,NaN,0,0,17,130315,0,827445,1187303,0
50041,0000 0109 FREDERICK ST0000,05E,1270 073,9,A,1900,5,0,15,3,4,RH3,D,0.0,0.0,4745,0,3920.0,NaN,NaN,1000.0,1.00000,NaN,NaN,0,0,17,0,0,469698,872300,360
197372,0000 0258 ARCH ST0000,03G,7087 036,42,D,1940,1,2,4,1,1,RH1,D,0.0,0.0,1000,0,2500.0,R,NaN,1000.0,1.00000,NaN,NaN,0,0,17,970730,0,135074,202612,0


In [8]:
#looking at average land value

In [9]:
sheet.RP1LNDVAL.describe()

count    2.080410e+05
mean     5.400424e+05
std      3.242311e+06
min      0.000000e+00
25%      8.076200e+04
50%      2.748560e+05
75%      5.276410e+05
max      5.835724e+08
Name: RP1LNDVAL, dtype: float64

In [10]:
sheet.RP1LNDVAL.mean()

540042.3986714158

In [11]:
#Figuring out how to merge RP1PRCLID, the parcel number, into a single 7 digit identifier
sheet.RP1PRCLID.describe()

count       208041
unique      208041
top       2506 002
freq             1
Name: RP1PRCLID, dtype: object

In [12]:
sheet['parcelno'] = sheet['RP1PRCLID']

In [13]:
sheet.head()

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0001 001
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0002 001
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0004 002
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0005 001
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0006 001


In [14]:
sheet[['Block','Lot']] = sheet.parcelno.str.split(" ",expand=True,)
sheet

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno,Block,Lot
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0001 001,0001,001
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0002 001,0002,001
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0004 002,0004,002
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0005 001,0005,001
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0006 001,0006,001
5,0000 0000 0000,08H,0007 001,1,G,1900,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,113437.50,R,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0007 001,0007,001
6,0000 2801 LEAVENWORTH ST0000,08H,0010 001,1,C1,1907,0,0,0,3,0,C2,C,0.0,0.0,133675,0,66124.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,110523,0,9087359,13631039,0,0010 001,0010,001
7,0495 0455 JEFFERSON ST0000,08H,0010 002,1,H,1907,0,0,252,4,0,C2,NaN,0.0,0.0,203840,0,47131.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,0,0,0,1,0,0010 002,0010,002
8,0000 2800 LEAVENWORTH ST0000,08H,0011 007,1,HC,0,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,55687.50,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,947696,32668516,32668516,67032,0011 007,0011,007
9,0000 0500 BEACH ST0000,08H,0011 008,1,C1,1974,7,0,36,4,29,C2,A,75.0,0.0,198525,0,57750.00,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,0,16694570,32668516,0,0011 008,0011,008


In [15]:
sheet["parcelno2"] = sheet["Block"] + sheet["Lot"]

In [16]:
sheet

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno,Block,Lot,parcelno2
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0001 001,0001,001,0001001
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0002 001,0002,001,0002001
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0004 002,0004,002,0004002
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0005 001,0005,001,0005001
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0006 001,0006,001,0006001
5,0000 0000 0000,08H,0007 001,1,G,1900,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,113437.50,R,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0007 001,0007,001,0007001
6,0000 2801 LEAVENWORTH ST0000,08H,0010 001,1,C1,1907,0,0,0,3,0,C2,C,0.0,0.0,133675,0,66124.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,110523,0,9087359,13631039,0,0010 001,0010,001,0010001
7,0495 0455 JEFFERSON ST0000,08H,0010 002,1,H,1907,0,0,252,4,0,C2,NaN,0.0,0.0,203840,0,47131.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,0,0,0,1,0,0010 002,0010,002,0010002
8,0000 2800 LEAVENWORTH ST0000,08H,0011 007,1,HC,0,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,55687.50,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,947696,32668516,32668516,67032,0011 007,0011,007,0011007
9,0000 0500 BEACH ST0000,08H,0011 008,1,C1,1974,7,0,36,4,29,C2,A,75.0,0.0,198525,0,57750.00,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,0,16694570,32668516,0,0011 008,0011,008,0011008


In [17]:
#Cleaning up column names

sheet['zoning'] = sheet['ZONE']
sheet['buildingsf'] = sheet['SQFT']
sheet['landarea'] = sheet['LAREA']
sheet['landvalue'] = sheet['RP1LNDVAL']

In [18]:
sheet

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno,Block,Lot,parcelno2,zoning,buildingsf,landarea,landvalue
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0001 001,0001,001,0001001,P,0,27965.00,0
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0002 001,0002,001,0002001,C2,0,8359.00,0
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0004 002,0004,002,0004002,C2,0,0.00,0
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0005 001,0005,001,0005001,C2,0,68153.00,0
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0006 001,0006,001,0006001,C2,0,108015.00,0
5,0000 0000 0000,08H,0007 001,1,G,1900,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,113437.50,R,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0007 001,0007,001,0007001,C2,0,113437.50,0
6,0000 2801 LEAVENWORTH ST0000,08H,0010 001,1,C1,1907,0,0,0,3,0,C2,C,0.0,0.0,133675,0,66124.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,110523,0,9087359,13631039,0,0010 001,0010,001,0010001,C2,133675,66124.00,13631039
7,0495 0455 JEFFERSON ST0000,08H,0010 002,1,H,1907,0,0,252,4,0,C2,NaN,0.0,0.0,203840,0,47131.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,0,0,0,1,0,0010 002,0010,002,0010002,C2,203840,47131.00,1
8,0000 2800 LEAVENWORTH ST0000,08H,0011 007,1,HC,0,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,55687.50,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,947696,32668516,32668516,67032,0011 007,0011,007,0011007,C2,0,55687.50,32668516
9,0000 0500 BEACH ST0000,08H,0011 008,1,C1,1974,7,0,36,4,29,C2,A,75.0,0.0,198525,0,57750.00,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,0,16694570,32668516,0,0011 008,0011,008,0011008,C2,198525,57750.00,32668516


In [19]:
#creating a new and clean data frame to work with
reldata = sheet[['parcelno2', 'zoning', 'buildingsf', 'landarea', 'landvalue']].copy()

In [20]:
reldata

,parcelno2,zoning,buildingsf,landarea,landvalue
0,0001001,P,0,27965.00,0
1,0002001,C2,0,8359.00,0
2,0004002,C2,0,0.00,0
3,0005001,C2,0,68153.00,0
4,0006001,C2,0,108015.00,0
5,0007001,C2,0,113437.50,0
6,0010001,C2,133675,66124.00,13631039
7,0010002,C2,203840,47131.00,1
8,0011007,C2,0,55687.50,32668516
9,0011008,C2,198525,57750.00,32668516


In [21]:
reldata.to_csv(r'AssessorDataNew.csv')

In [22]:
#bringing existing parcel data set to merge assessor data with

In [23]:
parcelzoning = pd.read_csv('parcelwzoning.csv', low_memory=False)

In [24]:
parcelzoning.sample(n=8)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning
40127,40127,40103,1760050,6058,3943,MIXRES,1759,2354.978107,NCD,True,MULTIPOLYGON (((-122.46264099909845 37.7640221...,NCD
123936,123936,123924,6325001A,1231,0,RESIDENT,0,1999.507050,RH-1,True,MULTIPOLYGON (((-122.4211014348048 37.71005711...,RH-1
5176,5176,5160,0257011,13200,6761,MIXRES,0,3580.060053,C-3-G,True,MULTIPOLYGON (((-122.40610099068319 37.7915456...,C-3-G
140317,140317,140313,0100046,1127,0,RESIDENT,0,1907.722972,RM-1,True,MULTIPOLYGON (((-122.41365595412655 37.8005063...,RM-1
137428,137428,137419,4953014,0,0,VACANT,0,4974.997288,NaN,True,MULTIPOLYGON (((-122.3729706505566 37.70932474...,NaN
148915,148915,148913,4950022,0,0,VACANT,0,4974.992278,NaN,True,MULTIPOLYGON (((-122.37762873223728 37.7121385...,NaN
4890,4890,4876,0275005,30204,0,RESIDENT,0,6721.846623,RC-4,True,MULTIPOLYGON (((-122.41315903198993 37.7896994...,RC-4
62640,62640,62614,2719051,4000,0,RESIDENT,0,2322.383292,RH-2,True,MULTIPOLYGON (((-122.44603950141732 37.7586059...,RH-2


In [25]:
parcelzoning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155468 entries, 0 to 155467
Data columns (total 12 columns):
Unnamed: 0     155468 non-null int64
objectid       155468 non-null int64
blklot         155468 non-null object
bldgsf         155468 non-null int64
totuses        155468 non-null int64
landuse        155468 non-null object
pdr            155468 non-null int64
parcelarea     155468 non-null float64
zoning_code    154673 non-null object
active         155451 non-null object
geometry       155468 non-null object
zoning         154673 non-null object
dtypes: float64(1), int64(5), object(6)
memory usage: 14.2+ MB


In [26]:
#merging data, tried two different ways to see what would preserve the most non-null data

In [27]:
combined_parcel = pd.merge(left=parcelzoning, right=reldata, how='left', left_on='blklot', right_on='parcelno2')

In [28]:
combined_parcel.sample(n=8)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_x,parcelno2,zoning_y,buildingsf,landarea,landvalue
65630,65630,65606,2450027,1184,0,RESIDENT,0,2975.808819,RH-1,True,MULTIPOLYGON (((-122.50482254113797 37.7370197...,RH-1,2450027,RH1,1184.0,3000.00,512618.0
104315,104315,104298,4789002B,758,0,RESIDENT,0,1764.809350,RH-1,True,MULTIPOLYGON (((-122.38690504321318 37.7292616...,RH-1,4789002B,RH1,758.0,1777.00,10525.0
47908,47908,47884,2001002,1120,4468,MIXRES,0,2777.191044,RH-1,True,MULTIPOLYGON (((-122.50776465071871 37.7542786...,RH-1,2001002,RH1,1120.0,2800.00,128677.0
67136,67136,67112,2461A021,1400,2147,MIXRES,2147,1858.346883,RH-1,True,MULTIPOLYGON (((-122.49328389413179 37.7382189...,RH-1,NaN,NaN,NaN,NaN,NaN
120572,120572,120560,7100003,2882,0,RESIDENT,0,2645.693857,RH-1,True,MULTIPOLYGON (((-122.44861042994371 37.7108550...,RH-1,7100003,RH1,2882.0,2665.00,529210.0
58954,58954,58930,2372048,6316,0,RESIDENT,0,5801.636631,RM-1,True,MULTIPOLYGON (((-122.50153307420622 37.7420226...,RM-1,2372048,RH1,6316.0,5749.00,927712.0
36538,36538,36513,1604023,1180,0,RESIDENT,0,2081.554077,RH-2,True,MULTIPOLYGON (((-122.50118112258441 37.7752888...,RH-2,1604023,RH2,1180.0,1999.00,271960.0
85076,85076,85062,3574099,4325,0,RESIDENT,0,3267.269752,RH-3,True,MULTIPOLYGON (((-122.41623657079732 37.7625204...,RH-3,3574099,RH3,4325.0,3291.58,236274.0


In [29]:
combined_parcel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155468 entries, 0 to 155467
Data columns (total 17 columns):
Unnamed: 0     155468 non-null int64
objectid       155468 non-null int64
blklot         155468 non-null object
bldgsf         155468 non-null int64
totuses        155468 non-null int64
landuse        155468 non-null object
pdr            155468 non-null int64
parcelarea     155468 non-null float64
zoning_code    154673 non-null object
active         155451 non-null object
geometry       155468 non-null object
zoning_x       154673 non-null object
parcelno2      145546 non-null object
zoning_y       140671 non-null object
buildingsf     145546 non-null float64
landarea       145546 non-null float64
landvalue      145546 non-null float64
dtypes: float64(4), int64(5), object(8)
memory usage: 21.4+ MB


In [30]:
#second merged data variable

In [31]:
combined_assessor = pd.merge(left=reldata, right=parcelzoning, how='left', left_on='parcelno2', right_on='blklot')

In [32]:
combined_assessor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208041 entries, 0 to 208040
Data columns (total 17 columns):
parcelno2      190328 non-null object
zoning_x       172651 non-null object
buildingsf     208041 non-null int64
landarea       208041 non-null float64
landvalue      208041 non-null int64
Unnamed: 0     145546 non-null float64
objectid       145546 non-null float64
blklot         145546 non-null object
bldgsf         145546 non-null float64
totuses        145546 non-null float64
landuse        145546 non-null object
pdr            145546 non-null float64
parcelarea     145546 non-null float64
zoning_code    144769 non-null object
active         145546 non-null object
geometry       145546 non-null object
zoning_y       144769 non-null object
dtypes: float64(7), int64(2), object(8)
memory usage: 28.6+ MB


In [33]:
combined_assessor.sample(n=8)

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y
71480,1805053,RM1,0,2622.0,0,42255.0,42231.0,1805053,0.0,0.0,VACANT,0.0,2590.166948,RM-1,True,MULTIPOLYGON (((-122.5087233397082 37.76019055...,RM-1
58075,1511068,NaN,859,0.0,356490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151694,5067019,RH1,5510,4030.0,761904,87299.0,87296.0,5067019,5510.0,0.0,RESIDENT,0.0,4016.458020,RH-1,True,MULTIPOLYGON (((-122.39950452871014 37.7145966...,RH-1
106373,2984010,RH1D,1663,3659.0,39189,75582.0,75562.0,2984010,1663.0,0.0,RESIDENT,0.0,3581.623752,RH-1(D),True,MULTIPOLYGON (((-122.46275130714515 37.7413215...,RH-1(D)
131089,3750673,NaN,906,0.0,385103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199954,7143020,RH1NC2,800,3746.0,117741,124189.0,124173.0,7143020,800.0,0.0,RESIDENT,0.0,3904.594699,NCD|RH-1,True,MULTIPOLYGON (((-122.45190720842508 37.7094954...,NCD|RH-1
18455,0481012,RH3,3800,2809.0,76623,10317.0,10286.0,0481012,3800.0,0.0,RESIDENT,0.0,2789.565834,RH-3,True,MULTIPOLYGON (((-122.42854319787531 37.8018919...,RH-3
182651,6545040,RH2,1809,0.0,499821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
combined_assessor.loc[combined_assessor.zoning_x=='PDR2']

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y
138477,3821009,PDR2,0,13358.00,39989,92464.0,92458.0,3821009,0.0,0.0,VACANT,0.0,13381.795069,PDR-1-D,True,MULTIPOLYGON (((-122.39876573712424 37.7664213...,PDR-1-D
146248,4288004,PDR2,31546,52200.00,3802482,94166.0,94157.0,4288004,31104.0,6292.0,MIXED,3368.0,49446.269364,PDR-2,True,MULTIPOLYGON (((-122.39348424412417 37.7506985...,PDR-2
146249,4288005,PDR2,64000,81237.00,8175338,94167.0,94158.0,4288005,64000.0,19932.0,PDR,17932.0,85616.458300,PDR-2,True,MULTIPOLYGON (((-122.39354920559127 37.7500063...,PDR-2
146250,4288006,PDR2,43200,64800.00,3591233,94168.0,94159.0,4288006,43200.0,55300.0,MIXED,10914.0,63526.786658,PDR-2,True,MULTIPOLYGON (((-122.39462070874185 37.7499444...,PDR-2
146253,4290011,PDR2,9300,9361.00,838063,94327.0,94318.0,4290011,9300.0,4443.0,MIXED,0.0,9578.749982,PDR-2,True,MULTIPOLYGON (((-122.39259612029906 37.7504734...,PDR-2
146630,4343001B,PDR2,35114,69996.00,7767079,103486.0,103464.0,4343001B,35114.0,5168.0,PDR,5168.0,69618.193281,PDR-2,True,MULTIPOLYGON (((-122.397193308955 37.749182371...,PDR-2
146631,4343001D,PDR2,19232,133297.00,3728545,103487.0,103465.0,4343001D,19232.0,2246.0,RETAIL/ENT,0.0,128261.904609,PDR-2,True,MULTIPOLYGON (((-122.40166401819543 37.7470196...,PDR-2
146646,4343030,PDR2,6834,17484.00,929495,94343.0,94334.0,4343030,6834.0,0.0,PDR,0.0,19108.892097,PDR-2,True,MULTIPOLYGON (((-122.40169703643947 37.7470185...,PDR-2
146650,4346003,PDR2,74122,343882.00,34173315,94192.0,94183.0,4346003,74122.0,0.0,PDR,0.0,343542.118835,PDR-2,True,MULTIPOLYGON (((-122.39372432462824 37.7496763...,PDR-2
146653,NaN,PDR2,108793,181269.00,20415781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#make new zoning column as a string data type based on existing zoning_x column

In [36]:
combined_assessor['finzone'] = combined_assessor['zoning_x'].astype(str)

In [37]:
combined_assessor.head()

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y,finzone
0,0001001,P,0,27965.0,0,147396.0,147394.0,0001001,0.0,0.0,PDR,0.0,28268.231998,P,True,MULTIPOLYGON (((-122.4210628988343 37.80860124...,P,P
1,0002001,C2,0,8359.0,0,72.0,73.0,0002001,0.0,0.0,PDR,0.0,8920.300030,C-2,True,MULTIPOLYGON (((-122.41981482898801 37.8087616...,C-2,C2
2,0004002,C2,0,0.0,0,6.0,7.0,0004002,0.0,20509.0,VACANT,0.0,31183.392188,C-2,True,MULTIPOLYGON (((-122.41379081959624 37.8085666...,C-2,C2
3,0005001,C2,0,68153.0,0,7.0,8.0,0005001,0.0,43725.0,PDR,0.0,69014.270707,C-2,True,MULTIPOLYGON (((-122.41593580428093 37.8082939...,C-2,C2
4,0006001,C2,0,108015.0,0,8.0,9.0,0006001,0.0,28728.0,PDR,0.0,106916.225315,C-2,True,MULTIPOLYGON (((-122.41758109128742 37.8080868...,C-2,C2


In [38]:
#filter data based on zoning column to only include PDR districts

In [39]:
filteredPDR3 = combined_assessor[combined_assessor['finzone'].str.contains("PDR")]

In [40]:
filteredPDR3.sample(n=14)

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y,finzone
159462,5596015,PDR2,5100,5209.00,636443,92026.0,92015.0,5596015,5100.0,0.0,PDR,0.0,4927.257927,PDR-2,True,MULTIPOLYGON (((-122.40688218485242 37.7410772...,PDR-2,PDR2
152177,5203083,PDR2,0,25968.00,2701317,103431.0,103409.0,5203083,0.0,0.0,VACANT,0.0,25399.639818,PDR-2,True,MULTIPOLYGON (((-122.38609844476173 37.7454182...,PDR-2,PDR2
139287,3980009,PDR1G,0,32224.00,4415625,85024.0,85010.0,3980009,0.0,0.0,VACANT,0.0,32407.226038,PDR-1-G,True,MULTIPOLYGON (((-122.40073091256717 37.7644461...,PDR-1-G,PDR1G
145089,4228035,PDR1G,1904,10000.00,582826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR1G
159442,NaN,PDR2,1860,3741.00,571081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR2
146250,4288006,PDR2,43200,64800.00,3591233,94168.0,94159.0,4288006,43200.0,55300.0,MIXED,10914.0,63526.786658,PDR-2,True,MULTIPOLYGON (((-122.39462070874185 37.7499444...,PDR-2,PDR2
148479,4786073,PDR2,5000,2500.00,303668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR2
140085,4015004,PDR1G,24033,19998.00,2406515,95929.0,95923.0,4015004,24033.0,23300.0,MIPS,0.0,20120.978550,PDR-1-G,True,MULTIPOLYGON (((-122.40862708071916 37.7623170...,PDR-1-G,PDR1G
159496,5607001,PDR-2,8200,7557.00,1298711,149540.0,149538.0,5607001,8200.0,4051.0,MIPS,0.0,7452.411930,PDR-2,True,MULTIPOLYGON (((-122.40615413914479 37.7391339...,PDR-2,PDR-2
148476,4786070,PDR2,2500,2500.00,344412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR2


In [41]:
filteredPDR3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 115261 to 161031
Data columns (total 18 columns):
parcelno2      178 non-null object
zoning_x       194 non-null object
buildingsf     194 non-null int64
landarea       194 non-null float64
landvalue      194 non-null int64
Unnamed: 0     139 non-null float64
objectid       139 non-null float64
blklot         139 non-null object
bldgsf         139 non-null float64
totuses        139 non-null float64
landuse        139 non-null object
pdr            139 non-null float64
parcelarea     139 non-null float64
zoning_code    139 non-null object
active         139 non-null object
geometry       139 non-null object
zoning_y       139 non-null object
finzone        194 non-null object
dtypes: float64(7), int64(2), object(9)
memory usage: 28.8+ KB


In [42]:
filteredPDR3.landarea.sum()

5130638.09

In [43]:
filteredPDR3.parcelarea.sum()

3979176.367905817

In [44]:
filteredPDR3.to_csv(r'filteredpdr3.csv')

In [45]:
#playing around with which zoning column is most reliable to filter off of

In [46]:
combined_parcel['finzone2'] = combined_parcel['zoning_y'].astype(str)

In [47]:
combined_parcel.head()

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_x,parcelno2,zoning_y,buildingsf,landarea,landvalue,finzone2
0,0,1,0026013,5460,0,RESIDENT,0,3864.061762,RH-3,True,MULTIPOLYGON (((-122.42108072666898 37.8047751...,RH-3,0026013,RH3,5460.0,3850.00,2756014.0,RH3
1,1,2,0026014,4500,3471,MIXRES,0,3361.025351,RH-3,True,MULTIPOLYGON (((-122.42117903244349 37.8047629...,RH-3,0026014,RH3,4500.0,3953.13,2857168.0,RH3
2,2,3,0026015,5783,0,RESIDENT,0,3272.033196,RH-3,True,MULTIPOLYGON (((-122.42126451502638 37.8047522...,RH-3,0026015,RH3,5783.0,3436.00,3863671.0,RH3
3,3,4,0026016,3700,0,RESIDENT,0,4570.767667,RH-3,True,MULTIPOLYGON (((-122.42134572244261 37.8047421...,RH-3,0026016,RH3,3700.0,4591.00,889887.0,RH3
4,4,5,0026017,4543,0,RESIDENT,0,5225.823735,RH-3,True,MULTIPOLYGON (((-122.4215252467804 37.80471971...,RH-3,0026017,RH3,4543.0,5248.00,1018543.0,RH3


In [48]:
combined_parcel.landarea.sum()

716457661.6999999

In [49]:
combined_parcel.landvalue.sum()

87217542264.0

In [50]:
#filtering for PDR parcels

In [51]:
filteredPDR4 = combined_parcel[combined_parcel['finzone2'].str.contains("PDR")]

In [52]:
filteredPDR4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139 entries, 69876 to 155456
Data columns (total 18 columns):
Unnamed: 0     139 non-null int64
objectid       139 non-null int64
blklot         139 non-null object
bldgsf         139 non-null int64
totuses        139 non-null int64
landuse        139 non-null object
pdr            139 non-null int64
parcelarea     139 non-null float64
zoning_code    139 non-null object
active         139 non-null object
geometry       139 non-null object
zoning_x       139 non-null object
parcelno2      139 non-null object
zoning_y       139 non-null object
buildingsf     139 non-null float64
landarea       139 non-null float64
landvalue      139 non-null float64
finzone2       139 non-null object
dtypes: float64(4), int64(5), object(9)
memory usage: 20.6+ KB


In [53]:
filteredpdrorig = pd.read_csv('filteredpdr.csv', low_memory=False)

In [54]:
filteredpdrorig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448 entries, 0 to 1447
Data columns (total 12 columns):
Unnamed: 0     1448 non-null int64
objectid       1448 non-null int64
blklot         1448 non-null object
bldgsf         1448 non-null int64
totuses        1448 non-null int64
landuse        1448 non-null object
pdr            1448 non-null int64
parcelarea     1448 non-null float64
zoning_code    1448 non-null object
active         1448 non-null bool
geometry       1448 non-null object
zoning         1448 non-null object
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 125.9+ KB


In [55]:
filteredPDR4.sample(n=10)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_x,parcelno2,zoning_y,buildingsf,landarea,landvalue,finzone2
95258,95258,95246,4247003,19200,15000,PDR,15000,19831.527654,PDR-1-G,True,MULTIPOLYGON (((-122.38898017998429 37.7536816...,PDR-1-G,4247003,PDR1G,19200.0,20000.0,1962923.0,PDR1G
92464,92464,92458,3821009,0,0,VACANT,0,13381.795069,PDR-1-D,True,MULTIPOLYGON (((-122.39876573712424 37.7664213...,PDR-1-D,3821009,PDR2,0.0,13358.0,39989.0,PDR2
94166,94166,94157,4288004,31104,6292,MIXED,3368,49446.269364,PDR-2,True,MULTIPOLYGON (((-122.39348424412417 37.7506985...,PDR-2,4288004,PDR2,31546.0,52200.0,3802482.0,PDR2
90192,90192,90188,5318046,0,3973,PDR,3973,55611.544325,PDR-2|PDR-1-B,True,MULTIPOLYGON (((-122.39889118245442 37.7389808...,PDR-2|PDR-1-B,5318046,PDR2,0.0,55695.0,3140678.0,PDR2
87693,87693,87694,5369070,10168,0,PDR,0,9038.367645,PDR-2,True,MULTIPOLYGON (((-122.40393816195196 37.7371051...,PDR-2,5369070,PDR2,10168.0,8930.0,1145923.0,PDR2
104424,104424,104426,5237012,13224,14039,PDR,12039,9896.589795,PDR-2,True,MULTIPOLYGON (((-122.38442047149 37.7405595769...,PDR-2,5237012,PDR2,13224.0,10000.0,705591.0,PDR2
95353,95353,95341,4830016,92440,0,PDR,0,106505.420500,PDR-2|PDR-1-B,True,MULTIPOLYGON (((-122.38991770991845 37.7261259...,PDR-2|PDR-1-B,4830016,PDR1B,92440.0,106500.0,6744805.0,PDR1B
95354,95354,95342,4830018,0,0,VACANT,0,2008.746022,PDR-2,True,MULTIPOLYGON (((-122.38948664960883 37.7262860...,PDR-2,4830018,PDR1B,0.0,2025.0,128441.0,PDR1B
85015,85015,85001,3938001,103626,0,PDR,0,80427.742253,PDR-1-D,True,MULTIPOLYGON (((-122.40090324115202 37.7662908...,PDR-1-D,3938001,PDR1D,103626.0,79997.0,21746655.0,PDR1D
150054,150054,150052,5237034,15925,0,PDR,0,19950.635136,PDR-2,True,MULTIPOLYGON (((-122.38406163111495 37.7400171...,PDR-2,5237034,PDR2,15925.0,20000.0,1697664.0,PDR2


In [56]:
filteredPDR4.landvalue.describe()

count    1.390000e+02
mean     2.539137e+06
std      4.559735e+06
min      0.000000e+00
25%      5.039945e+05
50%      1.147196e+06
75%      2.572362e+06
max      3.417332e+07
Name: landvalue, dtype: float64

In [57]:
filteredPDR4.landvalue.mean()

2539136.9568345323

In [58]:
filteredPDR4.landvalue.sum()

352940037.0

In [59]:
#Filtering for PDR-1-D and PDR-1-G parcels

In [60]:
PDRDandG = filteredPDR4[(filteredPDR4['finzone2'].str.contains("PDR1D")) | (filteredPDR4['finzone2'].str.contains("PDR1G"))]

In [61]:
PDRDandG.landvalue.mean()

3556787.6153846155

In [62]:
PDRDandG.landvalue.sum()

184952956.0

In [63]:
PDRDandG.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 69876 to 154182
Data columns (total 18 columns):
Unnamed: 0     52 non-null int64
objectid       52 non-null int64
blklot         52 non-null object
bldgsf         52 non-null int64
totuses        52 non-null int64
landuse        52 non-null object
pdr            52 non-null int64
parcelarea     52 non-null float64
zoning_code    52 non-null object
active         52 non-null object
geometry       52 non-null object
zoning_x       52 non-null object
parcelno2      52 non-null object
zoning_y       52 non-null object
buildingsf     52 non-null float64
landarea       52 non-null float64
landvalue      52 non-null float64
finzone2       52 non-null object
dtypes: float64(4), int64(5), object(9)
memory usage: 7.7+ KB


In [64]:
PDRDandG.landarea.sum()

1313467.0

In [65]:
PDRDandG.sample(n=30)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_x,parcelno2,zoning_y,buildingsf,landarea,landvalue,finzone2
84578,84578,84564,3958001G,15000,19284,MIXED,1428,8469.251150,PDR-1-D,True,MULTIPOLYGON (((-122.40368410665329 37.7648060...,PDR-1-D,3958001G,PDR1D,15000.0,8500.0,3182484.0,PDR1D
95931,95931,95925,4015007,8749,20697,MIXED,10270,8717.753156,PDR-1-G,True,MULTIPOLYGON (((-122.40874600057018 37.7625854...,PDR-1-G,4015007,PDR1G,8749.0,8749.0,1167143.0,PDR1G
82996,82996,82979,3574114,7913,0,PDR,0,3879.684177,PDR-1-G,True,MULTIPOLYGON (((-122.41518349914048 37.7625565...,PDR-1-G,3574114,PDR1G,7913.0,3920.0,653663.0,PDR1G
93428,93428,93419,4232010,279450,46862,PDR,46862,476876.940680,PDR-1-G,True,MULTIPOLYGON (((-122.38139951982541 37.7556862...,PDR-1-G,4232010,PDR1G,279450.0,476324.0,2495349.0,PDR1G
92459,92459,92453,3968001,51270,9653,MIXED,0,24612.718504,PDR-1-G,True,MULTIPOLYGON (((-122.41237207404596 37.7635460...,PDR-1-G,3968001,PDR1G,51270.0,24900.0,6875541.0,PDR1G
85308,85308,85294,3926001,5043,0,MIXED,0,16845.334734,PDR-1-G,True,MULTIPOLYGON (((-122.41260022552292 37.7659612...,PDR-1-G,3926001,PDR1G,5043.0,16827.0,1901241.0,PDR1G
82011,82011,81995,3529020,1915,0,RETAIL/ENT,0,2868.561999,PDR-1-G,True,MULTIPOLYGON (((-122.41505143357837 37.7686596...,PDR-1-G,3529020,PDR1G,1915.0,2940.0,214702.0,PDR1G
85343,85343,85328,3926002,29502,0,MIXED,0,15902.647508,PDR-1-G,True,MULTIPOLYGON (((-122.41256513838024 37.7655914...,PDR-1-G,3926002,PDR1G,29502.0,18410.0,10773702.0,PDR1G
92463,92463,92457,3821008,0,0,RETAIL/ENT,0,23654.203345,PDR-1-D,True,MULTIPOLYGON (((-122.39862726743131 37.7664296...,PDR-1-D,3821008,PDR1D,0.0,23642.0,1697664.0,PDR1D
126059,126059,126048,5657009D,18833,2184,MIPS,0,22693.107574,PDR-1-G,True,MULTIPOLYGON (((-122.40880330105348 37.7403877...,PDR-1-G,5657009D,PDR1G,18833.0,22000.0,2278221.0,PDR1G


In [66]:
PDRDandG["farvalue"] = (PDRDandG["buildingsf"] / PDRDandG["landarea"])
computeFAR = PDRDandG[['farvalue','geometry','finzone2','parcelno2','landvalue','buildingsf','landarea']].copy()
computeFAR.head()

/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,farvalue,geometry,finzone2,parcelno2,landvalue,buildingsf,landarea
69876,0.000000,MULTIPOLYGON (((-122.41651291865504 37.7653523...,PDR1G,3552013,155228.0,0.0,13198.0
78845,0.845522,MULTIPOLYGON (((-122.41661724763837 37.7641329...,PDR1G,3571005,1633779.0,7931.0,9380.0
80978,2.139429,MULTIPOLYGON (((-122.41523054003258 37.7630529...,PDR1G,3574008,1448668.0,6552.0,3062.5
80979,1.956571,MULTIPOLYGON (((-122.41519180346006 37.7626441...,PDR1G,3574009,2419171.0,35952.0,18375.0
81296,1.646332,MULTIPOLYGON (((-122.41429255369673 37.7622301...,PDR1G,3573003,10562452.0,33195.0,20163.0


In [67]:
#filtering data for parcels less than 0.3 FAR

In [68]:
filteredFAR = computeFAR[computeFAR['farvalue'] < 0.3]
filteredFAR.landarea.sum()

144272.0

In [69]:
filteredFAR.describe()

,farvalue,landvalue,buildingsf,landarea
count,9.000000,9.000000e+00,9.000000,9.000000
mean,0.040933,2.414404e+06,636.666667,16030.222222
std,0.099661,3.574476e+06,1667.922285,11453.764739
min,0.000000,0.000000e+00,0.000000,1005.000000
25%,0.000000,1.552280e+05,0.000000,10000.000000
50%,0.000000,1.697664e+06,0.000000,14806.000000
75%,0.000000,1.901241e+06,0.000000,23642.000000
max,0.299697,1.119620e+07,5043.000000,32224.000000


In [70]:
filteredFAR.landvalue.mean()

2414403.777777778

In [71]:
filteredFAR.landvalue.sum()

21729634.0

In [72]:
filteredFAR

,farvalue,geometry,finzone2,parcelno2,landvalue,buildingsf,landarea
69876,0.000000,MULTIPOLYGON (((-122.41651291865504 37.7653523...,PDR1G,3552013,155228.0,0.0,13198.0
85024,0.000000,MULTIPOLYGON (((-122.40073091256717 37.7644461...,PDR1G,3980009,4415625.0,0.0,32224.0
85308,0.299697,MULTIPOLYGON (((-122.41260022552292 37.7659612...,PDR1G,3926001,1901241.0,5043.0,16827.0
89740,0.000000,MULTIPOLYGON (((-122.41131829433732 37.7676943...,PDR1G,3923007,11196200.0,0.0,14806.0
92463,0.000000,MULTIPOLYGON (((-122.39862726743131 37.7664296...,PDR1D,3821008,1697664.0,0.0,23642.0
94767,0.000000,MULTIPOLYGON (((-122.39907429199019 37.7695939...,PDR1G,3807021,1746180.0,0.0,31400.0
96015,0.068700,MULTIPOLYGON (((-122.39041128806198 37.7548826...,PDR1G,4228018,490212.0,687.0,10000.0
96353,0.000000,MULTIPOLYGON (((-122.39303370688805 37.7550501...,PDR1G,4224090,127284.0,0.0,1170.0
154182,0.000000,MULTIPOLYGON (((-122.39303374577946 37.7550505...,PDR1G,4224089,0.0,0.0,1005.0


In [73]:
#filtering data for parcels over 20,000 sf

In [74]:
largeparcel = filteredFAR[filteredFAR["landarea"] > 20000]

In [75]:
largeparcel.sum()

farvalue                                                      0
geometry      MULTIPOLYGON (((-122.40073091256717 37.7644461...
finzone2                                        PDR1GPDR1DPDR1G
parcelno2                                 398000938210083807021
landvalue                                           7.85947e+06
buildingsf                                                    0
landarea                                                  87266
dtype: object

In [76]:
largeparcel.landvalue.mean()

2619823.0

In [77]:
largeparcel.landvalue.sum()

7859469.0

In [78]:
#saving the data

In [79]:
filteredFAR.to_csv(r'filteredFAR.csv')